# Tutorial 1 venco.py

This tutorial showcases the general structure and workflow of venco.py, as well as some basic features of its 6 main classes:
- DataParser
- GridModeller
- FlexEstimator
- DiaryBuilder
- Aggregator
- PostProcessor

All tutorials run on a very small subset of data from the 2017 German national travel survey (Mobilität in Deutschland (MiD17)), which might result in profiles having uncommon shapes. As such, the calculations and the examples proposed throughout all tutorials have the mere goal to exemplify the modelling steps and guide the use throughout the structure of venco.py and do not aim at providing an accurate quantification of demand-side flexibility from EVs.

For a more detailed description of venco.py, you can refer to the documentation at https://dlr-ve.gitlab.io/esy/vencopy/vencopy/

## Setting up the working space

This section allows you to import all required Python packages for data input and manipulation. The function os.chdir(path) allows us to point Python towards the top most directory which contains all useful venco.py functions that are going to be used in the tutorials.
Additionally, we set and read in the input dataframe (here the MiD17) and load the necessary yaml file, which contains some configuration settings.

In [1]:
import time
from pathlib import Path

In [2]:
from vencopy.core.dataparsers import parse_data
from vencopy.core.gridmodellers import GridModeller
from vencopy.core.flexestimators import FlexEstimator
from vencopy.core.diarybuilders import DiaryBuilder
from vencopy.core.profileaggregators import ProfileAggregator
from vencopy.core.postprocessors import PostProcessor
from vencopy.utils.utils import load_configs, create_output_folders

start_time = time.time()

We will have a look more in detail at each config file and what you can specify within it for each class throughtout the tutorials. For the time being, it is enough to know that the config files specify configurations, variable namings and settings for the different classes. There is one config file for each class, a global config and a local configuration config to specify eventual file paths on your machine.


In [3]:
base_path = Path.cwd().parent / "vencopy"
configs = load_configs(base_path)

## _DataParser_ class

To be able to estimate EV electric consumption and flexibililty, the first step in the venco.py framework implies accessing a travel survey data set, such as the MiD. This is carried out through a parsing interface to the original database. In the parsing interface to the data set, three main operations are carried out: the read-in of the travel survey trip data, stored in .dta or .csv files, filtering and cleaning of the original raw data set and a set of variable replacement operations to allow the composition of travel diaries in a following step (in the DiaryBuilder class).


In order to have consistent entry data for all variables and for different data sets, all database entries are harmonised, which includes generating unified data types and consistent variable naming. The naming convention for the variables and their respective input type can be specified in the venco.py config files that have been loaded previously.

First off, we modify the localConfig and globalConfig files so that it point to the current working directory and to the database subset we will use to explain the different classes.


In [4]:
# Adapt relative paths in config for tutorials
configs['dev_config']['global']['relative_path']['parse_output'] = Path.cwd().parent / configs['dev_config']['global']['relative_path']['parse_output']
configs['dev_config']['global']['relative_path']['diary_output'] = Path.cwd().parent / configs['dev_config']['global']['relative_path']['diary_output']
configs['dev_config']['global']['relative_path']['grid_output'] = Path.cwd().parent / configs['dev_config']['global']['relative_path']['grid_output']
configs['dev_config']['global']['relative_path']['flex_output'] = Path.cwd().parent / configs['dev_config']['global']['relative_path']['flex_output']
configs['dev_config']['global']['relative_path']['aggregator_output'] = Path.cwd().parent / configs['dev_config']['global']['relative_path']['aggregator_output']
configs['dev_config']['global']['relative_path']['processor_output'] = Path.cwd().parent / configs['dev_config']['global']['relative_path']['processor_output']


# Set reference dataset
datasetID = 'MiD17'

# Modify the localPathConfig file to point to the .csv file in the sampling folder in the tutorials directory where the dataset for the tutorials lies.
configs['user_config']['global']['absolute_path'][datasetID] = Path.cwd() /'data_sampling'

# Similarly we modify the datasetID in the global config file
configs['dev_config']['global']['files'][datasetID]['trips_data_raw'] = datasetID + '.csv'

# We also modify the parseConfig by removing some of the columns that are normally parsed from the MiD, which are not available in our semplified test dataframe
del configs['dev_config']['dataparsers']['data_variables']['household_id']
del configs['dev_config']['dataparsers']['data_variables']['person_id']

In [5]:
create_output_folders(configs=configs)

We can now run the first class and parse the dataset with the collection of mobility patterns into a more useful form for our scope.

In [6]:
data = parse_data(configs=configs)
data.process()

Generic file parsing properties set up.
Starting to retrieve local data file from c:\Users\jagm_li\Documents\vencopy_internal\vencopy\tutorials\data_sampling\MiD17.csv.
Finished loading 2124 rows of raw data of type .csv.
Running in debug mode.
Finished harmonization of variables.
Finished harmonization of ID variables.
Starting filtering, applying 8 filters.
All filters combined yielded that a total of 857 trips are taken into account.
This corresponds to 40.34839924670433 percent of the original data.
Completed park timestamp adjustments.
From 11791.33 km total mileage in the dataset after filtering, 0.0 % were cropped because they corresponded to split-trips from overnight trips.
Finished activity composition with 857 trips and 854 parking activites.
Parsing MiD dataset completed.


,index,unique_id,park_id,trip_id,is_driver,household_person_id,trip_weight,trip_scale_factor,trip_purpose,trip_distance,...,timestamp_start,timestamp_end,is_first_activity,is_last_activity,activity_id,next_activity_id,previous_activity_id,is_first_trip,is_first_park_activity,time_delta
0,1926,3,0.0,NaN,True,3,3.430376,919.415872,6,NaN,...,2017-04-08 00:00:00,2017-04-08 13:49:00,True,False,0.0,2.0,NaN,False,True,0 days 13:49:00
1,1927,3,NaN,2.0,True,3,3.430376,919.415872,6,15.20,...,2017-04-08 13:49:00,2017-04-08 14:15:00,False,False,2.0,2.0,0.0,True,False,0 days 00:26:00
2,1928,3,2.0,NaN,True,3,3.430376,919.415872,6,NaN,...,2017-04-08 14:15:00,2017-04-08 14:20:00,False,False,2.0,3.0,2.0,False,False,0 days 00:05:00
3,1929,3,NaN,3.0,True,3,3.430376,919.415872,8,15.20,...,2017-04-08 14:20:00,2017-04-08 14:40:00,False,False,3.0,3.0,2.0,False,False,0 days 00:20:00
4,1930,3,3.0,NaN,True,3,3.430376,919.415872,8,NaN,...,2017-04-08 14:40:00,2017-04-09 00:00:00,False,True,3.0,NaN,3.0,False,False,0 days 09:20:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1975,1103,2075,4.0,NaN,True,2075,0.905471,242.686011,5,NaN,...,2016-10-02 14:05:00,2016-10-02 14:25:00,False,False,4.0,5.0,4.0,False,False,0 days 00:20:00
1976,1104,2075,NaN,5.0,True,2075,0.905471,242.686011,4,6.18,...,2016-10-02 14:25:00,2016-10-02 14:35:00,False,False,5.0,5.0,4.0,False,False,0 days 00:10:00
1977,1105,2075,5.0,NaN,True,2075,0.905471,242.686011,4,NaN,...,2016-10-02 14:35:00,2016-10-02 14:45:00,False,False,5.0,6.0,5.0,False,False,0 days 00:10:00
1978,1106,2075,NaN,6.0,True,2075,0.905471,242.686011,8,11.40,...,2016-10-02 14:45:00,2016-10-02 15:05:00,False,False,6.0,6.0,5.0,False,False,0 days 00:20:00


## _GridModeller_ class

The charging infrastructure allocation makes use of a basic charging infrastructure model, which assumes the availability of charging stations when vehicles are parked. Since the analytical focus of the framework lies on a regional level (NUTS1-NUTS0), the infrastructure model is kept simple in the current version.


Charging availability is allocated based on a binary True–False mapping to a respective trip purpose in the venco.py config. Thus, different scenarios describing different charging availability scenarios, e.g., at home or at home and at work etc. can be distinguished, but neither a regional differentiation nor a charging availability probability or distribution are assumed.

At the end of the execution of the GridModeller class, a column representing the available charging power is added to the activities dataset.

In [7]:
grid = GridModeller(configs=configs, activities=data.activities)
grid.assign_grid()

Starting with charge connection replacement of location purposes.
Grid connection assignment complete.


## _FlexEstimator_ class

The flexEstimator class is the final class that is used to estimate the charging flexibility based on driving profiles and charge connection shares.
There are three integral inputs to the flexibililty estimation:
- A profile describing driven distances for each vehicle
- A profiles describing the available charging power if a vehicle is connected to the grid
- Techno–economic input assumptions

In [8]:
flex = FlexEstimator(configs=configs, activities=grid.activities)
flex.estimate_technical_flexibility_through_iteration()

Starting maximum battery level calculation.
Calculating maximum battery level for first activities.
Calculating maximum battery level for act 1.
Calculating maximum battery level for act 2.


C:\Users\jagm_li\Documents\vencopy_internal\vencopy\vencopy\core\flexestimators.py:244: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  first_park_activities.loc[:, "max_battery_level_end_unlimited"] = (
C:\Users\jagm_li\Documents\vencopy_internal\vencopy\vencopy\core\flexestimators.py:247: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  first_park_activities.loc[:, "max_battery_level_end"] = first_park_activities.loc[:,
C:\Users\jagm_li\Documents\vencopy_internal\vencopy\vencopy\core\flexestimators.py:253: FutureWarning: In a future version, `df.iloc[:,

Calculating maximum battery level for act 3.
Calculating maximum battery level for act 4.
Calculating maximum battery level for act 5.


C:\Users\jagm_li\Documents\vencopy_internal\vencopy\vencopy\core\flexestimators.py:343: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  indeces_trip_activities.loc[multi_index_trip, "max_battery_level_start"] = indeces_previous_park_activities.loc[
C:\Users\jagm_li\Documents\vencopy_internal\vencopy\vencopy\core\flexestimators.py:346: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  indeces_trip_activities.loc[multi_index_trip, "max_battery_level_end_unlimited"] = (
C:\Users\jagm_li\Documents\vencopy_internal\vencopy\vencopy\core\flexestimators.py:350: Fu

Calculating maximum battery level for act 6.
Calculating maximum battery level for act 7.
Calculating maximum battery level for act 8.


...

C:\Users\jagm_li\Documents\vencopy_internal\vencopy\vencopy\core\flexestimators.py:361: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  indeces_trip_activities.loc[multi_index_trip, "max_residual_need"] = res.where(
C:\Users\jagm_li\Documents\vencopy_internal\vencopy\vencopy\core\flexestimators.py:361: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  indeces_trip_activities.loc[multi_index_trip, "max_residual_need"] = res.where(
C:\Users\jagm_li\Documents\vencopy_internal\vencopy\vencopy\core\flexestimators.py:361: FutureWarning: In a future version, `df.

Calculating maximum battery level for act 12.


C:\Users\jagm_li\Documents\vencopy_internal\vencopy\vencopy\core\flexestimators.py:361: FutureWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  indeces_trip_activities.loc[multi_index_trip, "max_residual_need"] = res.where(


Finished iteration 1 / 10. Delta max battery level is 0, delta min battery level is 5 and threshold epsilon is 1.
Starting minimum battery level calculation.
Calculate minimum battery level for last activities.
Calculate minimum battery level for act 12.
Calculate minimum battery level for act 11.
Calculate minimum battery level for act 10.
Calculate minimum battery level for act 9.
Calculate minimum battery level for act 8.
Calculate minimum battery level for act 7.
Calculate minimum battery level for act 6.
Calculate minimum battery level for act 5.
Calculate minimum battery level for act 4.
Calculate minimum battery level for act 3.
Calculate minimum battery level for act 2.
Calculate minimum battery level for act 1.
Calculate minimum battery level for act 0.
Finished iteration 2 / 10. Delta max battery level is 0, delta min battery level is 0 and threshold epsilon is 1.
Technical flexibility estimation ended.


,unique_id,index,park_id,trip_id,is_driver,household_person_id,trip_weight,trip_scale_factor,trip_purpose,trip_distance,...,min_battery_level_end_unlimited,max_residual_need,min_residual_need,max_overshoot,min_undershoot,auxiliary_fuel_need,drain,max_charge_volume,min_battery_level_start_unlimited,residual_need
0,3,1926,0.0,NaN,True,3,3.430376,919.415872,6,NaN,...,None,None,None,136.785,0,NaN,NaN,136.785,-129.813,NaN
1,3,1927,NaN,2.0,True,3,3.430376,919.415872,6,15.20,...,None,0.0,0,None,None,NaN,2.7360,NaN,6.972,NaN
2,3,1928,2.0,NaN,True,3,3.430376,919.415872,6,NaN,...,None,None,None,0.0,2.736,NaN,NaN,0.000,4.236,NaN
3,3,1929,NaN,3.0,True,3,3.430376,919.415872,8,15.20,...,None,0.0,0,None,None,NaN,2.7360,NaN,4.236,NaN
4,3,1930,3.0,NaN,True,3,3.430376,919.415872,8,NaN,...,None,None,None,86.928,None,NaN,NaN,92.400,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1622,2075,1103,4.0,NaN,True,2075,0.905471,242.686011,5,NaN,...,None,None,None,0.4776,0,NaN,NaN,3.300,1.3644,NaN
1623,2075,1104,NaN,5.0,True,2075,0.905471,242.686011,4,6.18,...,None,0.0,0,None,None,NaN,1.1124,NaN,4.6644,NaN
1624,2075,1105,5.0,NaN,True,2075,0.905471,242.686011,4,NaN,...,None,None,None,0.0,2.052,NaN,NaN,0.000,3.552,NaN
1625,2075,1106,NaN,6.0,True,2075,0.905471,242.686011,8,11.40,...,None,0.0,0,None,None,NaN,2.0520,NaN,3.552,NaN


## _DiaryBuilder_ class

In the DiaryBuilder, individual trips at the survey day are consolidated into person-specific travel diaries comprising multiple trips.


The daily travel diary composition consists of three main steps: reformatting the database, allocating trip purposes and merging the obtained dataframe with other relevant variables from the original database.


In the first step, reformatting, the time dimension is transferred from the raw data (usually in minutes) to the necessary output format (e.g., hours). Each trip is split into shares, which are then assigned to the respective hour in which they took place, generating an hourly dataframe with a timestamp instead of a dataframe containing single trip entries.


Similarly, miles driven and the trip purpose are allocated to their respective hour and merged into daily travel diaries. Trips are assumed to determine the respective person’s stay in the consecutive hours up to the next trip and therefore are related to the charging availability between two trips. Trip purposes included in surveys may comprise trips carried out for work or education reasons, trips returning to home, trips to shopping facilities and other leisure activities. Currently, trips whose purpose is not specified are allocated to trips returning to their own household.

In [9]:
diary = DiaryBuilder(configs=configs, activities=flex.activities)
diary.create_diaries()

164 activities dropped from 1627 total activities because activity length equals zero.
Starting to discretise drain.
Discretisation finished for drain.
Needed time to discretise drain: 0.6019935607910156.
Starting to discretise available_power.
Discretisation finished for available_power.
Needed time to discretise available_power: 0.562082052230835.
Starting to discretise uncontrolled_charging.
Discretisation finished for uncontrolled_charging.
Needed time to discretise uncontrolled_charging: 0.5830378532409668.
Starting to discretise max_battery_level_start.
Discretisation finished for max_battery_level_start.
Needed time to discretise max_battery_level_start: 0.6380393505096436.
Starting to discretise min_battery_level_end.
Discretisation finished for min_battery_level_end.
Needed time to discretise min_battery_level_end: 0.5879738330841064.
Needed time to discretise all columns: 2.984130382537842.


## _ProfileAggregator_ class

In [10]:

profiles = ProfileAggregator(configs=configs, activities=diary.activities, profiles=diary)
profiles.aggregate_profiles()

Starting to aggregate drain to fleet level based on day of the week.
Dataset written to c:\Users\jagm_li\Documents\vencopy_internal\vencopy\output\profileaggregator\vencopy_output_profileaggregator_None_drain_MiD17.csv.
Aggregation finished for drain.
Needed time to aggregate drain: 0.02924323081970215.
Starting to aggregate charging_power to fleet level based on day of the week.
Dataset written to c:\Users\jagm_li\Documents\vencopy_internal\vencopy\output\profileaggregator\vencopy_output_profileaggregator_None_charging_power_MiD17.csv.
Aggregation finished for charging_power.
Needed time to aggregate charging_power: 0.05075788497924805.
Starting to aggregate uncontrolled_charging to fleet level based on day of the week.
Dataset written to c:\Users\jagm_li\Documents\vencopy_internal\vencopy\output\profileaggregator\vencopy_output_profileaggregator_None_uncontrolled_charging_MiD17.csv.
Aggregation finished for uncontrolled_charging.
Needed time to aggregate uncontrolled_charging: 0.0219

## _PostProcessor_ class

In [11]:
post = PostProcessor(configs=configs, profiles=profiles)
post.create_annual_profiles()
post.normalise()

Dataset written to c:\Users\jagm_li\Documents\vencopy_internal\vencopy\output\postprocessor\vencopy_output_postprocessor_annual_None_drain_MiD17.csv.
Run finished.
Dataset written to c:\Users\jagm_li\Documents\vencopy_internal\vencopy\output\postprocessor\vencopy_output_postprocessor_annual_None_uncontrolled_charging_MiD17.csv.
Run finished.
Dataset written to c:\Users\jagm_li\Documents\vencopy_internal\vencopy\output\postprocessor\vencopy_output_postprocessor_annual_None_charging_power_MiD17.csv.
Run finished.
Dataset written to c:\Users\jagm_li\Documents\vencopy_internal\vencopy\output\postprocessor\vencopy_output_postprocessor_annual_None_max_battery_level_MiD17.csv.
Run finished.
Dataset written to c:\Users\jagm_li\Documents\vencopy_internal\vencopy\output\postprocessor\vencopy_output_postprocessor_annual_None_min_battery_level_MiD17.csv.
Run finished.
Dataset written to c:\Users\jagm_li\Documents\vencopy_internal\vencopy\output\postprocessor\vencopy_output_postprocessor_normalized

In [12]:
elapsed_time = time.time() - start_time
print(f"Elapsed time: {elapsed_time}.")

Elapsed time: 9.273112297058105.


## Next Steps

In the next tutorials, you will learn more in detail the internal workings of each class and how to customise some settings.